# SELECT within SELECT

In [ ]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@localhost/sqlzoo' % (pwd))

## 1. Bigger than Russia

**List each country name where the population is larger than that of 'Russia'.**

```
world(name, continent, area, population, gdp)
```

## 2. Richer than UK

**Show the countries in Europe with a per capita GDP greater than 'United Kingdom'.**

> _Per Capita GDP_   
> The per capita GDP is the gdp/population

## 3. Neighbours of Argentina and Australia


List the name and continent of countries in the continents containing either Argentina or Australia. Order by name of the country.

## 4. Between Canada and Poland

Which country has a population that is more than Canada but less than Poland? Show the name and the population.

## 5. Percentages of Germany

Germany (population 80 million) has the largest population of the countries in Europe. Austria (population 8.5 million) has 11% of the population of Germany.

**Show the name and the population of each country in Europe. Show the population as a percentage of the population of Germany.**

The format should be Name, Percentage for example:

name	| percentage
--------|-----------
Albania	| 3%
Andorra	| 0%
Austria	| 11%
...	| ...

> _Decimal places_   
> You can use the function ROUND to remove the decimal places.

> _Percent symbol %_
> You can use the function CONCAT to add the percentage symbol.

[To get a well rounded view of the important features of SQL you should move on to the next tutorial concerning aggregates.](https://sqlzoo.net/wiki/SUM_and_COUNT)

To gain an absurdly detailed view of one insignificant feature of the language, read on.

We can use the word `ALL` to allow >= or > or < or <=to act over a list. For example, you can find the largest country in the world, by population with this query:

```sql
SELECT name
  FROM world
 WHERE population >= ALL(SELECT population
                           FROM world
                          WHERE population>0)
```

You need the condition **population>0** in the sub-query as some countries have **null** for population.

## 6. Bigger than every country in Europe

Which countries have a GDP greater than every country in Europe? [Give the name only.] (Some countries may have NULL gdp values)

We can refer to values in the outer SELECT within the inner SELECT. We can name the tables so that we can tell the difference between the inner and outer versions.

## 7. Largest in each continent

**Find the largest country (by area) in each continent, show the continent, the name and the area:**

```sql
SELECT continent, name, population FROM world x
  WHERE population >= ALL
    (SELECT population FROM world y
        WHERE y.continent=x.continent
          AND population>0)
```

> __The above example is known as a correlated or synchronized sub-query.__   
> 
> Using correlated subqueries
> A correlated subquery works like a nested loop: the subquery only has access to rows related to a single > record at a time in the outer query. The technique relies on table aliases to identify two different uses of the same table, one in the outer query and the other in the subquery.
> 
> One way to interpret the line in the **WHERE** clause that references the two table is _“… where the correlated values are the same”._
> 
> In the example provided, you would say _“select the country details from world where the population is greater than or equal to the population of all countries where the continent is the same”._

## 8. First country of each continent (alphabetically)

**List each continent and the name of the country that comes first alphabetically.**

## 9. Difficult Questions That Utilize Techniques Not Covered In Prior Sections

**Find the continents where all countries have a population <= 25000000. Then find the names of the countries associated with these continents. Show name, continent and population.**

## 10.

**Some countries have populations more than three times that of any of their neighbours (in the same continent). Give the countries and continents.**